In [11]:
import os
import matplotlib.pyplot as plt
import pandas as pd
import sklearn

In [12]:
from sklearn.neighbors import KNeighborsClassifier

In [5]:
%matplotlib widget

## Load data

In [6]:
subjects = [s.split(".")[0] for s in os.listdir(".") if s.startswith("S")]
subjects.sort()
subjects

['S10',
 'S11',
 'S13',
 'S14',
 'S15',
 'S16',
 'S17',
 'S2',
 'S3',
 'S4',
 'S5',
 'S6',
 'S7',
 'S8',
 'S9']

In [7]:
selected_subject = subjects[4]

In [8]:
data = pd.read_csv(f"{selected_subject}.csv")

In [9]:
data.head()

,Unnamed: 0,chest_ECG,chest_EMG,chest_EDA,chest_Temp,chest_Resp,chest_ACC1,chest_ACC2,chest_ACC3,wrist_BVP,wrist_EDA,wrist_TEMP,wrist_ACC1,wrist_ACC2,wrist_ACC3,label,subject
0,0,-0.025543,-0.016159,1.362228,33.493042,0.691223,0.8296,0.3122,1.1474,1.35,0.220298,29.79,40.0,-45.0,127.0,0,S15
1,1,-0.012955,-0.014145,1.357651,33.447693,0.694275,0.8310,0.2970,1.2390,1.35,0.220298,29.79,40.0,-45.0,127.0,0,S15
2,2,0.002518,-0.013000,1.345444,33.480957,0.695801,0.8450,0.2610,1.2450,1.35,0.220298,29.79,40.0,-45.0,127.0,0,S15
3,3,0.008011,-0.003204,1.353073,33.499084,0.689697,0.8670,0.2066,1.1598,1.35,0.220298,29.79,40.0,-45.0,127.0,0,S15
4,4,0.007141,-0.000687,1.344299,33.493042,0.682068,0.9002,0.1642,1.0302,1.35,0.220298,29.79,40.0,-45.0,127.0,0,S15


## Models

In [13]:
classifier = KNeighborsClassifier(n_neighbors=5)

In [15]:
data.columns

Index(['Unnamed: 0', 'chest_ECG', 'chest_EMG', 'chest_EDA', 'chest_Temp',
       'chest_Resp', 'chest_ACC1', 'chest_ACC2', 'chest_ACC3', 'wrist_BVP',
       'wrist_EDA', 'wrist_TEMP', 'wrist_ACC1', 'wrist_ACC2', 'wrist_ACC3',
       'label', 'subject'],
      dtype='object')

In [16]:
data_columns = [
    'chest_ECG', 'chest_EMG', 'chest_EDA', 'chest_Temp',
    'chest_Resp', 'chest_ACC1', 'chest_ACC2', 'chest_ACC3', 'wrist_BVP',
    'wrist_EDA', 'wrist_TEMP', 'wrist_ACC1', 'wrist_ACC2', 'wrist_ACC3'
]

In [17]:
target_column = "label"

In [18]:
X = data[data_columns]

In [20]:
y = data[target_column]

In [21]:
classifier.fit(X, y)

KNeighborsClassifier()

In [33]:
print(classifier.predict(X.iloc[0:10]))

[0 0 0 0 0 0 0 0 0 0]


In [34]:
predicts = classifier.predict(X)

In [50]:
print("Accuracy:", (sum(predicts == y)/len(y))*100, "%")

Accuracy: 99.98231966053748 %


In [47]:
fig, ax = plt.subplots()
ax.plot(y)
ax.plot(predicts)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [48]:
fig, ax = plt.subplots()
ax.plot(predicts - y)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …